In [ ]:
from PyQt5.QtGui import *
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
import sys
import cv2
import random
import numpy as np
import base64
import io
from PIL import Image

import importer
import os
import svm
import numpy as npk
import joblib



class MainWindow(QTabWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('手写数字识别')
        self.initUI()
    
    
    def initUI(self):
        main_widget = QWidget()
        main_layout = QHBoxLayout()
        font = QFont('楷体', 15)
        
        left_widget = QWidget()
        left_layout = QVBoxLayout()
        img_title = QLabel("测试样本")
        img_title.setFont(font)
        img_title.setAlignment(Qt.AlignCenter)
        self.img_label = QLabel()
        
        img_path="wdn.png"
        self.img_label.setPixmap(QPixmap(img_path))
        self.img_label.setScaledContents(True)
        left_layout.addWidget(img_title)
        left_layout.addWidget(self.img_label, 1, Qt.AlignCenter)
        # left_layout.setAlignment(Qt.AlignCenter)
        left_widget.setLayout(left_layout)

        right_widget = QWidget()
        right_layout = QVBoxLayout()
        btn_change = QPushButton(" 加载测试样本 ")
        btn_change.clicked.connect(self.change_img)
        btn_change.setFont(font)
        btn_predict = QPushButton(" 识别手写字体 ")
        #btn_predict.setFont(font)
       #btn_predict.clicked.connect(self.predict_img)

        label_result = QLabel(' 识 别 结 果 ')
        self.result = QLabel("7")
        label_result.setFont(QFont('楷体', 16))
        self.result.setFont(QFont('楷体', 24))
        right_layout.addStretch()
        right_layout.addWidget(label_result, 0, Qt.AlignCenter)
        right_layout.addStretch()
        right_layout.addWidget(self.result, 0, Qt.AlignCenter)
        right_layout.addStretch()
        right_layout.addWidget(btn_change)
        right_layout.addWidget(btn_predict)
        right_layout.addStretch()
        # right_layout.addSpacing(5)
        right_widget.setLayout(right_layout)
        
       
        label_super = QLabel()
        label_super.setFont(QFont('楷体', 12))
        label_super.setOpenExternalLinks(True)
        label_super.setAlignment(Qt.AlignRight)

        main_layout.addWidget(left_widget)
        main_layout.addWidget(right_widget)
        main_widget.setLayout(main_layout)
        self.addTab(main_widget, '主页面')

    def change_img(self):
        index_1 = random.randint(0, 9)
        index_2 = random.randint(0, 900)
        path = "D:\jupyterPJ\svm"
        tbasePath = os.path.join(path, "data\\Mnist-image\\test\\")
        tcName = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
        testPath = tbasePath + tcName[index_1]
        tflist = svm.get_file_list(testPath) 
        tdataMat, tdataLabel = read_and_convert(tflist[index_2])
        Filestr = tflist[index_2]
        img = cv2.imread(tflist[index_2])
        print(index_1)
        img = cv2.resize(img, (400, 400))
        cv2.imwrite('wdn.png', img)
        self.img_label.setPixmap(QPixmap('wdn.png'))
        self.predict_img(tflist, index_2)
        
    def predict_img(self, tflist, index):
        path = "D:\jupyterPJ\svm"
        model_path=os.path.join(path,'model\\svm.model')
        clf = joblib.load(model_path)
        tdataMat, tdataLabel = read_and_convert(tflist[index])
        preResult = clf.predict(tdataMat)
        self.result.setText(preResult[0])
        

        
if __name__ == "__main__":
    app = QApplication(sys.argv)
    app.aboutToQuit.connect(app.deleteLater)
    x = MainWindow()
    x.show()
    app.exec_()

1
7
7
9
6
0
9
0
9
5
9
2
1
7
8
5
8
4
4
4
7
2
3
6
8
4
0
4
7
2
9
7
8
4
0
4
8
7
8
5
0
7


In [3]:
def read_and_convert(file):
    dataLabel = []
    dataM =  np.zeros(400)
    imgNameStr = file
    imgName = get_img_name_str(imgNameStr)  # 得到 数字_实例编号.png
    # print("imgName: {}".format(imgName))
    classTag = imgName.split(".")[0].split("_")[0]  # 得到 类标签(数字)
    # print("classTag: {}".format(classTag))
    dataLabel.append(classTag)
    dataMat = img2vector(imgNameStr)
    return dataMat, dataLabel

In [4]:
def get_img_name_str(imgPath):
    return imgPath.split(os.path.sep)[-1]

In [5]:
def img2vector(imgFile):
    # print("in img2vector func--para:{}".format(imgFile))
    img = Image.open(imgFile).convert('L')
    img_arr = np.array(img, 'i')  # 20px * 20px 灰度图像q 
    img_normalization = np.round(img_arr / 255)  # 对灰度值进行归一化
    img_arr2 = np.reshape(img_normalization, (1, -1))  # 1 * 400 矩阵
    return img_arr2